# Task 2: Training Models

## Data Cleaning of the Datasets

Mount the google drive folder

In [52]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive/NLP_Project'
  sys.path.append(path_to_project)
  IN_COLAB = True
except:
  IN_COLAB = False

### Dataset cleaning process

The aim of this section is to create a single dataset using the two datasets at our disposal to simplify the retrieval of the information. Specifically we are interested in the descriptions of the problems, the various examples for the solutions and the respective time and space complexities. The aim is to simplify the retrieval of the information contained in the datasets. We then proceed to eliminate the na values, fix the index, and eliminate the comments from the code solutions.


First, the datasets problem_and_human_solutions_list and complexity_labels_light are downloaded and turned into DataFrames.




In [53]:
from huggingface_hub import hf_hub_download
import json
import pandas as pd
import numpy as np

In [54]:
# Download the dataset from Hugging Face Hub
file_path_1 = hf_hub_download(
    repo_id="facebook/BigOBench",
    filename="data/problem_and_human_solutions_list.jsonl",
    repo_type="dataset"
)

# Read the JSONL file and convert it to a DataFrame
df_problem_and_human_solutions_list = []
with open(file_path_1, "r", encoding="utf-8") as f:
    for line in f:
        try:
            df_problem_and_human_solutions_list.append(json.loads(line))
        except json.JSONDecodeError as e:
            print("Erorr JSON:", e)

df_problem_and_human_solutions_list = pd.DataFrame(df_problem_and_human_solutions_list)

In [55]:
# Download the second dataset from Hugging Face Hub
file_path_2 = hf_hub_download(
    repo_id="facebook/BigOBench",
    filename="data/complexity_labels_light.jsonl",
    repo_type="dataset"
)

# Read the JSONL file and convert it to a DataFrame
df_complexity_labels_light = []
with open(file_path_2, "r", encoding="utf-8") as f:
    for line in f:
        try:
            df_complexity_labels_light.append(json.loads(line))
        except json.JSONDecodeError as e:
            print("Erorr JSON:", e)

df_complexity_labels_light = pd.DataFrame(df_complexity_labels_light)

In [ ]:
df_problem_and_human_solutions_list.head(5)

,problem_id,problem_name,description,correct_solution_list,data_source,source_specific_limits,codeforces_specific_metadata,tests,human_accuracy_rate,dataclass,complexity_framework
0,0,339_C. Xenia and Weights,{'text': 'Xenia has a set of weights and pan s...,"[{'solution_id': '0_0', 'solution_code': '__au...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 339, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '0000000101 3 ', '...",0.281633,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.40579710144927...
1,1,1547_E. Air Conditioners,{'text': 'On a strip of land of length n there...,"[{'solution_id': '1_0', 'solution_code': 'def ...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 1547, 'cf_index': 'E', 'cf_p...",{'public_tests': [{'input': '5 6 2 2 5 14 16 ...,0.620833,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.11409395973154...
2,2,268_C. Beautiful Sets of Points,{'text': 'Manao has invented a new mathematica...,"[{'solution_id': '2_0', 'solution_code': 'if _...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 268, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '2 2 ', 'output': ...",0.525066,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.05527638190954...
3,3,478_C. Table Decorations,"{'text': 'You have r red, g green and b blue b...","[{'solution_id': '3_0', 'solution_code': 'a = ...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 478, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '1 1 1 ', 'output'...",0.562264,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.41610738255033...
4,4,5_C. Longest Regular Bracket Sequence,{'text': 'This is yet another problem dealing ...,"[{'solution_id': '4_0', 'solution_code': 'stri...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 5, 'cf_index': 'C', 'cf_poin...","{'public_tests': [{'input': ')((())))(()()) ',...",0.395939,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.03846153846153...


In [ ]:
df_complexity_labels_light.head(5)

,problem_id,problem_name,solution_id,time_complexity_inferred,space_complexity_inferred,time_curve_coefficient,space_curve_coefficient
0,0,339_C. Xenia and Weights,0_0,O(1),O(n**2),0.000019,0.012433
1,0,339_C. Xenia and Weights,0_1,None,None,NaN,NaN
2,0,339_C. Xenia and Weights,0_2,O(1),O(1),0.000004,584.000000
3,0,339_C. Xenia and Weights,0_3,None,None,NaN,NaN
4,0,339_C. Xenia and Weights,0_4,O(1),O(1),0.000013,323.000000


The attribute 'correst_solution_list' of the first dataset contains a list of code solutions for each sample. These solutions are stored as dictionaries containing as keys 'solution_id' and 'solution_code'. The aim is to obtain two new columns (attributes) in the final dataset one containing the solution id, and the other containing one solution code per sample. 

The Datasets results too big to be managed with the resources at our disposal, so it was decided to reduce the size of the datasets reducing the number of solutions for each description. After many considerations, it was choosen 300000 as the maximum amount of total samples for the final dataset. The following code reduces to 100 the maximum number of solutions for each description.

In [ ]:
# reduce num of possible correct solutions in correct_solution_list
max = 100  # num of corresct solutions kept

for problem in df_problem_and_human_solutions_list.correct_solution_list:
    if len(problem) > max:
        problem[:] = problem[:max]

In [ ]:
# check num of solutions:
for problem in df_problem_and_human_solutions_list.correct_solution_list[:30]:
    print('Number of samples:', len(problem))

Number of samples: 69
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 64
Number of samples: 75
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 54
Number of samples: 100
Number of samples: 100
Number of samples: 85
Number of samples: 100
Number of samples: 75
Number of samples: 86
Number of samples: 100
Number of samples: 90
Number of samples: 78
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 98
Number of samples: 100
Number of samples: 91


Now we create a new DataFrame with columns 'solution_id' and 'solution_code' from the attribute 'correct_solution_list', and 'problem_description' from the atribute 'description' of the dataset df_problem_and_human_solutions_list.

In [ ]:
# modify correct_solutions_list to have two different attributes (solution_id and solution_code) insted of a list of them
solutions_list = df_problem_and_human_solutions_list.correct_solution_list.copy()
descriptions = df_problem_and_human_solutions_list.description.copy()

rows = []
for desc, solutions in zip(descriptions, solutions_list):
    for solution in solutions:
        rows.append({
            'problem_description': desc['text'],
            'solution_id': solution['solution_id'],
            'solution_code': solution['solution_code']
        })
new_matrix = pd.DataFrame(rows)

new_matrix.head(70)

,problem_description,solution_id,solution_code
0,Xenia has a set of weights and pan scales. Eac...,0_0,__author__ = 'ratnesh.mishra'\n\nweights = map...
1,Xenia has a set of weights and pan scales. Eac...,0_1,s = input()\nm = int(input())\nk1 = k2 = 0\nz ...
2,Xenia has a set of weights and pan scales. Eac...,0_2,import sys\nsys.setrecursionlimit (1000000)\n\...
3,Xenia has a set of weights and pan scales. Eac...,0_3,"from __future__ import division, print_functio..."
4,Xenia has a set of weights and pan scales. Eac...,0_4,# Target - Expert on CF\n# Be Humblefool\n\nim...
...,...,...,...
65,Xenia has a set of weights and pan scales. Eac...,0_65,"p, m = [i for i, x in enumerate(input(), 1) if..."
66,Xenia has a set of weights and pan scales. Eac...,0_66,"s=input()\nA=[]\nfor i in range(1,11):\n if..."
67,Xenia has a set of weights and pan scales. Eac...,0_67,"def go(i, d, last):\n if i>0 and d<=0 or d>..."
68,Xenia has a set of weights and pan scales. Eac...,0_68,from sys import setrecursionlimit\nsetrecursio...


In [ ]:
new_matrix.shape

(293586, 3)

In [ ]:
new_matrix.describe()

,problem_description,solution_id,solution_code
count,293586,293586,293586
unique,3031,293586,289745
top,Hilbert's Hotel is a very unusual hotel since ...,0_0,"from sys import stdin, stdout\ninput = stdin.b..."
freq,200,1,2


Now the Dataset new_matrix and the second dataset are merged to abtain one line for each solution and keep only the solutions present in the new_matrix

In [ ]:
new_matrix.head(5)

,problem_description,solution_id,solution_code
0,Xenia has a set of weights and pan scales. Eac...,0_0,__author__ = 'ratnesh.mishra'\n\nweights = map...
1,Xenia has a set of weights and pan scales. Eac...,0_1,s = input()\nm = int(input())\nk1 = k2 = 0\nz ...
2,Xenia has a set of weights and pan scales. Eac...,0_2,import sys\nsys.setrecursionlimit (1000000)\n\...
3,Xenia has a set of weights and pan scales. Eac...,0_3,"from __future__ import division, print_functio..."
4,Xenia has a set of weights and pan scales. Eac...,0_4,# Target - Expert on CF\n# Be Humblefool\n\nim...


In [ ]:
df_complexity_labels_light.head(5)

,problem_id,problem_name,solution_id,time_complexity_inferred,space_complexity_inferred,time_curve_coefficient,space_curve_coefficient
0,0,339_C. Xenia and Weights,0_0,O(1),O(n**2),0.000019,0.012433
1,0,339_C. Xenia and Weights,0_1,None,None,NaN,NaN
2,0,339_C. Xenia and Weights,0_2,O(1),O(1),0.000004,584.000000
3,0,339_C. Xenia and Weights,0_3,None,None,NaN,NaN
4,0,339_C. Xenia and Weights,0_4,O(1),O(1),0.000013,323.000000


In [ ]:
df = new_matrix.merge(df_complexity_labels_light, on='solution_id', how='left')

In [ ]:
df.head(5)

,problem_description,solution_id,solution_code,problem_id,problem_name,time_complexity_inferred,space_complexity_inferred,time_curve_coefficient,space_curve_coefficient
0,Xenia has a set of weights and pan scales. Eac...,0_0,__author__ = 'ratnesh.mishra'\n\nweights = map...,0,339_C. Xenia and Weights,O(1),O(n**2),0.000019,0.012433
1,Xenia has a set of weights and pan scales. Eac...,0_1,s = input()\nm = int(input())\nk1 = k2 = 0\nz ...,0,339_C. Xenia and Weights,None,None,NaN,NaN
2,Xenia has a set of weights and pan scales. Eac...,0_2,import sys\nsys.setrecursionlimit (1000000)\n\...,0,339_C. Xenia and Weights,O(1),O(1),0.000004,584.000000
3,Xenia has a set of weights and pan scales. Eac...,0_3,"from __future__ import division, print_functio...",0,339_C. Xenia and Weights,None,None,NaN,NaN
4,Xenia has a set of weights and pan scales. Eac...,0_4,# Target - Expert on CF\n# Be Humblefool\n\nim...,0,339_C. Xenia and Weights,O(1),O(1),0.000013,323.000000


In [ ]:
df.shape

(293586, 9)

Remove from df the atributes that we aren't going to use during the modeling

In [ ]:
df = df.drop(columns = ['problem_id', 'time_curve_coefficient', 'space_curve_coefficient'])

In [ ]:
df.head(5)

,problem_description,solution_id,solution_code,problem_name,time_complexity_inferred,space_complexity_inferred
0,Xenia has a set of weights and pan scales. Eac...,0_0,__author__ = 'ratnesh.mishra'\n\nweights = map...,339_C. Xenia and Weights,O(1),O(n**2)
1,Xenia has a set of weights and pan scales. Eac...,0_1,s = input()\nm = int(input())\nk1 = k2 = 0\nz ...,339_C. Xenia and Weights,None,None
2,Xenia has a set of weights and pan scales. Eac...,0_2,import sys\nsys.setrecursionlimit (1000000)\n\...,339_C. Xenia and Weights,O(1),O(1)
3,Xenia has a set of weights and pan scales. Eac...,0_3,"from __future__ import division, print_functio...",339_C. Xenia and Weights,None,None
4,Xenia has a set of weights and pan scales. Eac...,0_4,# Target - Expert on CF\n# Be Humblefool\n\nim...,339_C. Xenia and Weights,O(1),O(1)


From the preliminary analysis we know that only the second dataset has na values. The na values are now eliminated form df

In [ ]:
# how many values of df are nan
df[df.isna().any(axis=1)]

,problem_description,solution_id,solution_code,problem_name,time_complexity_inferred,space_complexity_inferred
1,Xenia has a set of weights and pan scales. Eac...,0_1,s = input()\nm = int(input())\nk1 = k2 = 0\nz ...,339_C. Xenia and Weights,None,None
3,Xenia has a set of weights and pan scales. Eac...,0_3,"from __future__ import division, print_functio...",339_C. Xenia and Weights,None,None
5,Xenia has a set of weights and pan scales. Eac...,0_5,import sys\nsys.setrecursionlimit(100000)\n\n\...,339_C. Xenia and Weights,None,None
7,Xenia has a set of weights and pan scales. Eac...,0_7,import sys\nsys.setrecursionlimit(1500)\ndef d...,339_C. Xenia and Weights,None,None
9,Xenia has a set of weights and pan scales. Eac...,0_9,s=input()\nm=int(input())\na=[]\nfor j in rang...,339_C. Xenia and Weights,None,None
...,...,...,...,...,...,...
293369,You are given an integer L. Construct a direct...,3102_83,import sys\ninput = sys.stdin.readline\n\nL = ...,p03267 AtCoder Beginner Contest 108 - All Your...,None,None
293422,You are given an integer sequence of length N....,3103_36,"from itertools import accumulate\nn, *A = map(...",p03739 AtCoder Beginner Contest 059 - Sequence,None,None
293431,You are given an integer sequence of length N....,3103_45,"n, *A = map(int, open(0).read().split())\ndef ...",p03739 AtCoder Beginner Contest 059 - Sequence,None,None
293446,You are given an integer sequence of length N....,3103_60,"n = int(input())\na = list(map(int,input().spl...",p03739 AtCoder Beginner Contest 059 - Sequence,None,None


In [ ]:
df = df.dropna()

In [ ]:
# size of the temp_df after removal of na values
df.shape

(244877, 6)

Rename the index

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.head(3)

,problem_description,solution_id,solution_code,problem_name,time_complexity_inferred,space_complexity_inferred
0,Xenia has a set of weights and pan scales. Eac...,0_0,__author__ = 'ratnesh.mishra'\n\nweights = map...,339_C. Xenia and Weights,O(1),O(n**2)
1,Xenia has a set of weights and pan scales. Eac...,0_2,import sys\nsys.setrecursionlimit (1000000)\n\...,339_C. Xenia and Weights,O(1),O(1)
2,Xenia has a set of weights and pan scales. Eac...,0_4,# Target - Expert on CF\n# Be Humblefool\n\nim...,339_C. Xenia and Weights,O(1),O(1)


### Clean the code solutions
In the code of the solutions are present many elements that would render the training of models harder, such as comments (both inline and multiple lines).

Apon inspection of the code solutions, one code solution resulted not correct sintatically, so it was decided to drop it

In [ ]:
df.solution_code[114411]

'A,B = map(int,input().split(" "))\n\ns1 = "####################################################################################################\\n\\\n.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#\\n"\ns2 = "....................................................................................................\\n\\\n#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.\\n"\nblack = \'#\' * 100 + \'\\n\'\nwhite = \'.\' * 100 + \'\\n\'\n\ncnt = (A-1) // 50\nmer = (A - 1) % 50\nif mer == 0:\n\tup = s1 * cnt + black\nelse:\n\tup = s1 * cnt + black + (\'.#\' * mer) + (\'#\' * (100 - 2*mer)) + \'\\n\' + black\n\ncnt = (B-1) // 50\nmer = (B - 1) % 50\n\nif mer == 0:\n\tdown = s2 * cnt + white\nelse:\n\tdown = s2 * cnt + white + (\'#.\' * mer) + (\'.\' * (100 - 2*mer)) + \'\\n\' + white\n\nprint(str(len((up+down)) // 100) + " 100")\nprint(up + down)\n\n'

In [ ]:
df = df.drop(df.index[114411], axis = 0)

In [ ]:
df[114409:114413]

,problem_description,solution_id,solution_code,problem_name,time_complexity_inferred,space_complexity_inferred
114409,You are given two integers A and B.\n\nPrint a...,1447_0,\nimport sys\nsys.setrecursionlimit(1 << 25)\n...,p03404 AtCoder Regular Contest 093 - Grid Comp...,O(1),O(n)
114410,You are given two integers A and B.\n\nPrint a...,1447_1,"from math import ceil\n\nA, B = map(int, input...",p03404 AtCoder Regular Contest 093 - Grid Comp...,O(n+m),O(n+m)
114412,You are given two integers A and B.\n\nPrint a...,1447_3,"a, b = map(int, input().split())\na -= 1\nb -=...",p03404 AtCoder Regular Contest 093 - Grid Comp...,O(nlogn),O(n**2)
114413,You are given two integers A and B.\n\nPrint a...,1447_4,"ab = list(map(int, input().split()))\n\ng = [[...",p03404 AtCoder Regular Contest 093 - Grid Comp...,O(1),O(1)


In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.shape

(244876, 6)

Visualize inline comments

In [ ]:
import re
import nltk
from nltk import FreqDist

inline_comment_regex = '([^\"|^\']\#[^\'|^\"|^\n]*\n{1})'
concatenated_code = ' '.join(df.solution_code)
inline_comments_in_code = re.findall(inline_comment_regex, concatenated_code)
counts = nltk.FreqDist(inline_comments_in_code)
counts

FreqDist({'\n#\n': 2906, ' #!/usr/bin/env python3\n': 1645, ' # -*- coding: utf-8 -*-\n': 1553, '\n# region fastio\n': 1209, ' #\n': 1012, ' #!/usr/bin/env python\n': 693, '\n# endregion\n': 683, '\n#     else:\n': 545, ' # cook your dish here\n': 540, '\n# ------------------------------\n': 436, ...})

In [ ]:
for solution in df.solution_code:
    comment_code = re.findall(inline_comment_regex, solution)

In [ ]:
# print an example of code solution that contains inline comments
df.solution_code[65669]

'n = int(input())\ncont = 0\nmatriz = []\nfor i in range(n):\n    linha = list(input())\n    matriz.append(linha)\n    for i in range(n):\n        if linha[i] == \'#\':\n            cont+=1\nif cont%5 != 0:\n    print("NO")\nelse:\n    for i in range(0,n-1):\n        for j in range(1,n):\n            if matriz[i][j] == \'#\' and i < n-2 and j < n-1: # topo da cruz\n                if matriz[i+1][j] != \'#\' or matriz[i+2][j] != \'#\' or matriz[i+1][j-1] != \'#\' or matriz[i+1][j+1]!= \'#\':\n                    continue\n    \n                else:\n                    matriz[i][j] = \'.\'\n                    matriz[i+1][j] = \'.\'\n                    matriz[i+2][j] = \'.\'\n                    matriz[i+1][j-1] = \'.\'\n                    matriz[i+1][j+1] = \'.\'\n                    cont -= 5\n    if cont == 0:\n        print("YES")\n    else:\n        print("NO")\n            \n \t   \t\t\t  \t\t \t \t \t \t\t\t\t\t\t\t'

Remove inline comments

In [ ]:
for solution in df.solution_code:
    solution = re.sub(inline_comment_regex, '\n', solution)

In [ ]:
# print the solution code now without inline comments
df.solution_code[65669]

'n = int(input())\ncont = 0\nmatriz = []\nfor i in range(n):\n    linha = list(input())\n    matriz.append(linha)\n    for i in range(n):\n        if linha[i] == \'#\':\n            cont+=1\nif cont%5 != 0:\n    print("NO")\nelse:\n    for i in range(0,n-1):\n        for j in range(1,n):\n            if matriz[i][j] == \'#\' and i < n-2 and j < n-1: # topo da cruz\n                if matriz[i+1][j] != \'#\' or matriz[i+2][j] != \'#\' or matriz[i+1][j-1] != \'#\' or matriz[i+1][j+1]!= \'#\':\n                    continue\n    \n                else:\n                    matriz[i][j] = \'.\'\n                    matriz[i+1][j] = \'.\'\n                    matriz[i+2][j] = \'.\'\n                    matriz[i+1][j-1] = \'.\'\n                    matriz[i+1][j+1] = \'.\'\n                    cont -= 5\n    if cont == 0:\n        print("YES")\n    else:\n        print("NO")\n            \n \t   \t\t\t  \t\t \t \t \t \t\t\t\t\t\t\t'

Consider now multi-line comments

In [ ]:
multi_comment_regex = '([\'\"])\1\1[\d\D]*?\1{3}'
concatenated_code = ' '.join(df.solution_code)
multi_comments_in_code = re.findall(multi_comment_regex, concatenated_code)
counts = nltk.FreqDist(multi_comments_in_code)
counts

FreqDist({})

In [ ]:
for solution in df.solution_code:
    comment_code = re.findall(multi_comment_regex, solution)
    if comment_code:
        print(df.loc[df['solution_code'] == solution].index[0])

There are no multiline comments

### Save final Dataset in storage

In [ ]:
saving_path = path_to_project + '/final_ds.csv' if IN_COLAB else './final_ds.csv'
df.to_csv(saving_path, sep=',', index=False, encoding='utf-8')

## Transformers for Text Generation, Training and Fine-Tuning
In this section we are going to train and evaluate Transformers models to generate text.

### Libraries

Connect to Goodle Drive (if needed), add testing variables to the environment, and upload the dataset from storage

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive/NLP_Project'
  sys.path.append(path_to_project)
  IN_COLAB = True
except:
  IN_COLAB = False

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
import numpy as np

In [ ]:
dataset_path = path_to_project + '/final_ds.csv' if IN_COLAB else './final_ds.csv'
df = pd.read_csv(dataset_path)

In [ ]:
df.head(3)

,problem_description,solution_id,solution_code,problem_name,time_complexity_inferred,space_complexity_inferred
0,Xenia has a set of weights and pan scales. Eac...,0_0,__author__ = 'ratnesh.mishra'\n\nweights = map...,339_C. Xenia and Weights,O(1),O(n**2)
1,Xenia has a set of weights and pan scales. Eac...,0_2,import sys\nsys.setrecursionlimit (1000000)\n\...,339_C. Xenia and Weights,O(1),O(1)
2,Xenia has a set of weights and pan scales. Eac...,0_4,import sys\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,339_C. Xenia and Weights,O(1),O(1)


### Train T5-Base

The developers of the Text-To-Text Transfer Transformer (T5) write:

'With T5, we propose reframing all NLP tasks into a unified text-to-text-format where the input and output are always text strings, in contrast to BERT-style models that can only output either a class label or a span of the input. Our text-to-text framework allows us to use the same model, loss function, and hyperparameters on any NLP task.'

The model has 223M parameters.

Import libraries that are going to be used in the following subsections

In [ ]:
!pip install -q transformers
!pip install datasets

In [ ]:
from datasets import Dataset, DatasetDict

#### Dataset Pre-processing



For the Model T5-Base only 8000 samples are kept from the dataset due to the limitation of the resources availabe to us. We decided to keep the training (and fine-tuning) time of the models at approximately 2 hours for this section, and the amount of samples allowed us to maintain this contraint.

In [ ]:
## only in initial part: reduce ds to allow faster testing of the code
df = df.head(8000).copy()
df.shape

(8000, 6)

Subset the data into training dataset, validation dataset and test dataset. We decided to use 10% of the dataset for tesing, and the remaining for train and validation.

In [ ]:
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(df, test_size=0.1)
train, val = train_test_split(train_val, test_size=0.2)

In [ ]:
print('# train instances: ', train.shape[0])
print('# test instances:  ', test.shape[0])
print('# val instances:   ', val.shape[0])

# train instances:  5760
# test instances:   800
# val instances:    1440


We check the device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#### Model and Tokenizer

Retrieve the model and the tokenizer

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
model_name = 'google-t5/t5-base'

Upload the Tokenizer

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

Print some useful information on the tokenizer

In [ ]:
print("vocabulary size: ", tokenizer.vocab_size)

vocabulary size:  32000


In [ ]:
list(tokenizer.get_vocab().items())[600:610]

[('▁big', 600),
 ('▁God', 601),
 ('▁dass', 602),
 ('im', 603),
 ('▁30', 604),
 ('▁event', 605),
 ('▁development', 606),
 ('▁form', 607),
 ('▁read', 608),
 ('▁hand', 609)]

In [ ]:
text = "You have an array in input, order the elements in it in O(n) time complexity. Add a wrong wordd"
encoded_input = tokenizer._tokenize(text)
print(encoded_input)

['▁You', '▁have', '▁an', '▁array', '▁in', '▁input', ',', '▁order', '▁the', '▁elements', '▁in', '▁it', '▁in', '▁O', '(', 'n', ')', '▁time', '▁complexity', '.', '▁Add', '▁', 'a', '▁wrong', '▁word', 'd']


In [ ]:
encoded_ids = tokenizer(text)['input_ids']
print(encoded_ids)

[148, 43, 46, 5590, 16, 3785, 6, 455, 8, 2479, 16, 34, 16, 411, 599, 29, 61, 97, 11641, 5, 2334, 3, 9, 1786, 1448, 26, 1]


The model is uploaded and connected to the device

In [ ]:
t5 = T5ForConditionalGeneration.from_pretrained(model_name, device_map=device)
t5.resize_token_embeddings(len(tokenizer))

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Embedding(32100, 768)

In [ ]:
print(t5)

T5ForConditionalGeneration(
  (shared): Embedding(32100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

Consider now the number of parameters of the model

In [ ]:
n_params = sum(param.numel() for param in t5.parameters())
n_params

222882048

In [ ]:
for name, param in t5.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Parameter shape: {param.size()}")
    print(f"Is trainable: {param.requires_grad}")
    print()

Parameter name: shared.weight
Parameter shape: torch.Size([32100, 768])
Is trainable: True

Parameter name: encoder.block.0.layer.0.SelfAttention.q.weight
Parameter shape: torch.Size([768, 768])
Is trainable: True

Parameter name: encoder.block.0.layer.0.SelfAttention.k.weight
Parameter shape: torch.Size([768, 768])
Is trainable: True

Parameter name: encoder.block.0.layer.0.SelfAttention.v.weight
Parameter shape: torch.Size([768, 768])
Is trainable: True

Parameter name: encoder.block.0.layer.0.SelfAttention.o.weight
Parameter shape: torch.Size([768, 768])
Is trainable: True

Parameter name: encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight
Parameter shape: torch.Size([32, 12])
Is trainable: True

Parameter name: encoder.block.0.layer.0.layer_norm.weight
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: encoder.block.0.layer.1.DenseReluDense.wi.weight
Parameter shape: torch.Size([3072, 768])
Is trainable: True

Parameter name: encoder.block.0.la

#### Training


Now that we have both Tokenizer and Model we can tokenize the dataset and train the model

We obtain a single string for each conversation of problem description and solution, where the user input and the system response are separated by eos tokens.

In [ ]:
def apply_eos_token(idx, df, eos_token):
    # build the user input including the problem description, the time complexity and the space complexity
    chat_string = 'User:' + df.loc[idx, 'problem_description'] + ' Time complexity: ' + df.loc[idx, 'time_complexity_inferred'] + '; Space complexity: ' + df.loc[idx, 'space_complexity_inferred']
    # now add the eos token and the response from the assistant, the code solution for the problem
    chat_string = chat_string + eos_token + 'Assistant: ' + df.loc[idx, 'solution_code'] + eos_token
    return chat_string

In [ ]:
train_str = [apply_eos_token(idx, train, tokenizer.eos_token) for idx in train.index]
test_str = [apply_eos_token(idx, test, tokenizer.eos_token) for idx in test.index]
val_str = [apply_eos_token(idx, val, tokenizer.eos_token) for idx in val.index]

In [ ]:
train_str[0]

'User:It is known that the area of a regular dodecagon inscribed in a circle of radius a is 3a^2.\n\nGiven an integer r, find the area of a regular dodecagon inscribed in a circle of radius r.\n\nConstraints\n\n* 1 \\leq r \\leq 100\n* r is an integer.\n\nInput\n\nInput is given from Standard Input in the following format:\n\n\nr\n\n\nOutput\n\nPrint an integer representing the area of the regular dodecagon.\n\nExamples\n\nInput\n\n4\n\n\nOutput\n\n48\n\n\nInput\n\n15\n\n\nOutput\n\n675\n\n\nInput\n\n80\n\n\nOutput\n\n19200 Time complexity: O(1); Space complexity: O(1)</s>Assistant: N = int(input())\n\nprint(3*(N**2))</s>'

The strings where the eos token was applied, are now put in a dictionary structure

In [ ]:
train_data = Dataset.from_dict({'chat': train_str})
test_data = Dataset.from_dict({'chat': test_str})
val_data = Dataset.from_dict({'chat': val_str})

In [ ]:
data = DatasetDict()
data['train'] = train_data
data['val'] = val_data
data['test'] = test_data

The data is tokenized

In [ ]:
def tokenize_function(examples):
    input_encodings = tokenizer(examples["chat"],
        truncation=True,
        padding="max_length",
        max_length=512)
    sample = {
        'input_ids': input_encodings.input_ids,
        'attention_mask': input_encodings.attention_mask,
        'labels': input_encodings.input_ids.copy()
    }
    return sample

tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/5760 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
# get all sequences in same batch
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

Now the training starts

In [ ]:
from transformers import TrainingArguments, Trainer
import os

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = TrainingArguments(
    "t5_trainer",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    save_steps=500,
    eval_steps=500,
    learning_rate=1e-4,       # t5 needs a higher lr than other models
    lr_scheduler_type="linear",
    bf16=True,
    report_to=None,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Trainer(
    model= t5,
    args= training_args,
    train_dataset= tokenized_data['train'],
    eval_dataset= tokenized_data['val'],
    data_collator= data_collator
)

Measure the time taken by the model to finish training

In [ ]:
import time
begin = time.time()

In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,0.153200
1000,0.004200


TrainOutput(global_step=1080, training_loss=0.07308270597347506, metrics={'train_runtime': 7709.3416, 'train_samples_per_second': 2.241, 'train_steps_per_second': 0.14, 'total_flos': 1.05227923488768e+16, 'train_loss': 0.07308270597347506, 'epoch': 3.0})

In [ ]:
end = time.time()
print("Training time: ", end - begin)

Training time:  7709.833685159683


In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(t5.named_parameters())

print('The T5 model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:2]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[2:14]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-2:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The T5 model has 257 different named parameters.

==== Embedding Layer ====

shared.weight                                           (32100, 768)
encoder.block.0.layer.0.SelfAttention.q.weight            (768, 768)

==== First Transformer ====

encoder.block.0.layer.0.SelfAttention.k.weight            (768, 768)
encoder.block.0.layer.0.SelfAttention.v.weight            (768, 768)
encoder.block.0.layer.0.SelfAttention.o.weight            (768, 768)
encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight     (32, 12)
encoder.block.0.layer.0.layer_norm.weight                     (768,)
encoder.block.0.layer.1.DenseReluDense.wi.weight         (3072, 768)
encoder.block.0.layer.1.DenseReluDense.wo.weight         (768, 3072)
encoder.block.0.layer.1.layer_norm.weight                     (768,)
encoder.block.1.layer.0.SelfAttention.q.weight            (768, 768)
encoder.block.1.layer.0.SelfAttention.k.weight            (768, 768)
encoder.block.1.layer.0.SelfAttention.v.weight      

In [ ]:
t5.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

Save model's parameters

In [ ]:
from datetime import datetime
t5_training_path = path_to_project + '/Transformer-trained-models/' + f"t5_train_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"
tokenizer.save_pretrained(t5_training_path)
t5.save_pretrained(t5_training_path)
print(f"Checkpoint saved at: \'{t5_training_path}\'")

Checkpoint saved at: '/content/drive/MyDrive/NLP_Project/Transformer-trained-models/t5_train_2025_05_18_12_35_46'


#### Testing

Retrieve the trained model

In [ ]:
device = 'cuda'

#t5_training_path = '/content/drive/MyDrive/NLP_Project/Transformer-trained-models/t5_train_2025_05_18_12_35_46'
tokenizer = T5Tokenizer.from_pretrained(t5_training_path)
t5 = T5ForConditionalGeneration.from_pretrained(t5_training_path, device_map=device)

To test the model, we first extract one conversation from the test data randomly, give it as input to the model, and we see the response

In [ ]:
import random

random.seed(43)

idx = random.choice(range(len(test_data))) # select a random conversation
print(idx)
dialogue = test_data['chat'][idx]
print(dialogue)

39
User: Vlad likes to eat in cafes very much. During his life, he has visited cafes n times. Unfortunately, Vlad started to feel that his last visits are not any different from each other. To fix that Vlad had a small research.

First of all, Vlad assigned individual indices to all cafes. Then, he wrote down indices of cafes he visited in a row, in order of visiting them. Now, Vlad wants to find such a cafe that his last visit to that cafe was before his last visits to every other cafe. In other words, he wants to find such a cafe that he hasn't been there for as long as possible. Help Vlad to find that cafe.

Input

In first line there is one integer n (1 ≤ n ≤ 2·105) — number of cafes indices written by Vlad.

In second line, n numbers a1, a2, ..., an (0 ≤ ai ≤ 2·105) are written — indices of cafes in order of being visited by Vlad. Vlad could visit some cafes more than once. Note that in numeration, some indices could be omitted.

Output

Print one integer — index of the cafe that 

In [ ]:
# now take only the 'input part' and the 'output part'
# parse string
test_input, test_output, en = dialogue.split('</s>')

# add eos token at the end of test_input and test_output
test_input = test_input + tokenizer.eos_token + ' Assistant: '
test_output = test_output + tokenizer.eos_token

print('User input: \n', test_input)
print('User input lenght: ', len(test_input))
print('####')
print('Correct solution output: \n', test_output)

User input: 
 User: Vlad likes to eat in cafes very much. During his life, he has visited cafes n times. Unfortunately, Vlad started to feel that his last visits are not any different from each other. To fix that Vlad had a small research.

First of all, Vlad assigned individual indices to all cafes. Then, he wrote down indices of cafes he visited in a row, in order of visiting them. Now, Vlad wants to find such a cafe that his last visit to that cafe was before his last visits to every other cafe. In other words, he wants to find such a cafe that he hasn't been there for as long as possible. Help Vlad to find that cafe.

Input

In first line there is one integer n (1 ≤ n ≤ 2·105) — number of cafes indices written by Vlad.

In second line, n numbers a1, a2, ..., an (0 ≤ ai ≤ 2·105) are written — indices of cafes in order of being visited by Vlad. Vlad could visit some cafes more than once. Note that in numeration, some indices could be omitted.

Output

Print one integer — index of the

We generate the response from the model

In [ ]:
input_ids = tokenizer(test_input, return_tensors="pt", max_length = 2000).to(device)
output = t5.generate(**input_ids, max_new_tokens=800)
gen_text = tokenizer.decode(output[0])

print(gen_text)
print(gen_text.split('Assistant:')[-1])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<pad> User: Vlad likes to eat in cafes very much. During his life, he has visited cafes n times. Unfortunately, Vlad started to feel that his last visits are not any different from each other. To fix that Vlad had a small research. First of all, Vlad assigned individual indices to all cafes. Then, he wrote down indices of cafes he visited in a row, in order of visiting them. Now, Vlad wants to find such a cafe that his last visit to that cafe was before his last visits to every other cafe. In other words, he wants to find such a cafe that he hasn't been there for as long as possible. Help Vlad to find that cafe. Input In first line there is one integer n (1 <unk> n <unk> 2<unk> 105) — number of cafes indices written by Vlad. In second line, n numbers a1, a2, ..., an (0 <unk> ai <unk> 2<unk> 105) are written — indices of cafes in order of being visited by Vlad. Vlad could visit some cafes more than once. Note that in numeration, some indices could be omitted. Output Print one integer — 

The model does not perform very well and struggles to produce code in output and to maintain the conversation format given to it. What it tries to do is to describe the solution process without explicitly generating code. 

We expected this type of reults, the T5 model has very good performance on translation and sumarization, while it wasn't trained as a chatbot to produce code initially. Moreover, due to our limited resources, 8000 samples are too little to train such a big model (223M parameters) in an efective way.

#### Evaluation Metrics

It was decided to consider the metrics Perplexity, BLEU and F1.

**Perplexity**

We select only the first three samples of the test set due to limitations in the RAM available to us

In [ ]:
# get inputs from test_data
test_input = [dialogue.split('</s>')[0] + tokenizer.eos_token + 'Assistant: ' for dialogue in test_data['chat'][:3]]

# get outputs from test data
test_output = [dialogue.split('</s>')[1] + tokenizer.eos_token for dialogue in test_data['chat'][:3]]

In [ ]:
!pip install lmppl

In [ ]:
import lmppl

scorer = lmppl.EncoderDecoderLM(t5_training_path)

ppl = scorer.get_perplexity(input_texts= test_input, output_texts=test_output)
print(list( ppl))
print(f"average perplexity: {sum(ppl)/len(ppl)}")

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/configuration_auto.py:1099: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:4056: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
  0%|          | 0/1 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Passing a tuple of `past_key_va

[23387.28944204467, 1260.3922689555861, 22569.483405842497]
average perplexity: 15739.055038947583


In [ ]:
print(f"user input: \n{test_input[ppl.index(min(ppl))]}")
print('#################')
print(f"prediction: \n{test_output[ppl.index(min(ppl))]}")

user input: 
User: Gleb ordered pizza home. When the courier delivered the pizza, he was very upset, because several pieces of sausage lay on the crust, and he does not really like the crust.

The pizza is a circle of radius r and center at the origin. Pizza consists of the main part — circle of radius r - d with center at the origin, and crust around the main part of the width d. Pieces of sausage are also circles. The radius of the i -th piece of the sausage is ri, and the center is given as a pair (xi, yi).

Gleb asks you to help determine the number of pieces of sausage caught on the crust. A piece of sausage got on the crust, if it completely lies on the crust.

Input

First string contains two integer numbers r and d (0 ≤ d < r ≤ 500) — the radius of pizza and the width of crust.

Next line contains one integer number n — the number of pieces of sausage (1 ≤ n ≤ 105).

Each of next n lines contains three integer numbers xi, yi and ri ( - 500 ≤ xi, yi ≤ 500, 0 ≤ ri ≤ 500), where x

**BLEU**

BLEU focuses on precision by counting matching n-grams.

In [ ]:
!pip -q install parlai

In [ ]:
from parlai.core.metrics import BleuMetric

input_ids = tokenizer(test_input[0], return_tensors="pt", max_length = 2000).to(device)
output = t5.generate(**input_ids, max_new_tokens=800)
gen_text = tokenizer.decode(output[0])

bleu = BleuMetric.compute(gen_text, [test_output[0]])
print(f"BLEU: {bleu}")

BLEU: 3.804e-09


The Bleu metric has a very low value, indicating very little overlap between the generated text and the solution of the test sample 

**F1 Score**

In [ ]:
from parlai.core.metrics import F1Metric

f1_score = F1Metric.compute(gen_text, [test_output[0]])
print(f"F1: {f1_score}")

F1: 0.03226


### Fine-tuning T5-small


**Discaimer:** We decided to not include the whole code for this section: the code was very similar to the one used to train the T5-base model and, given the already very long notebook and the not-excellent performance after the fine-tuning, we considered best to not include it.

The T5-small model has 60M parameters, and is part of the family of the T5 text-to-text models. It was trained on Common Crawl.
For the fine-tuning of T5-small we decided to keep 5000 samples. The model is relatively small, with only 60M parameters and already trained: a higher amount of samples led to worse performance because it caused the training to overwrite parameters unecessarily.

#### Model and Tokenizer
We include the model specifics:

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Model

In [ ]:
model_name = 'google-t5/t5-small'

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
t5 = T5ForConditionalGeneration.from_pretrained(model_name, device_map=device)
t5.resize_token_embeddings(len(tokenizer))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Embedding(32100, 512)

In [ ]:
print(t5)

T5ForConditionalGeneration(
  (shared): Embedding(32100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

#### Results

The perplexity calculated before the fine-tuning was: [66.54613901709006, 105.79112153990572, 35432.329456935135] with
average perplexity: 11868.222239164044.

The training lasted 1314.5914108753204 seconds.

We then tested the text generation of the fine-tuned model on a random sample from the test set: also in this case, the model does not perform well. It doesn't generate the code, mainly copies the input, and struggles to keep the separation between 'User' and 'Assistant'. Overall the generated text from T5-small is less clear than the text generated by the training of T5-base: the bigger amount of parameters of T5-base guarantees better text quality.

We considered the evaluation metrics Perplexity, Bleu and F1.

**Perplexity:** it was calculated considering three samples form the test set, the results are: [439.4691293912123, 720.7729464902942, 1981.2242516858746] with average perplexity: 1047.1554425224604. We can see that the fine-tuning did not increase considerably the performance of the model. The average of the perplexity decreased though, indicating a better understanding of the new functionality with respect to the model prior the fine-tuning.

**Bleu:** 7.417e-06

**F1-score:** 0.0744

Also Bleu and F1 scores are extremely low.

### Fine-tuning gpt2-small

GPT-2 is a transformers model pretrained on a very large corpus of English data in a self-supervised fashion. This means it was pretrained on the raw texts only, with no humans labelling them in any way (which is why it can use lots of publicly available data) with an automatic process to generate inputs and labels from those texts. More precisely, it was trained to guess the next word in sentences.

Moreover, inputs are sequences of continuous text of a certain length and the targets are the same sequence, shifted one token (word or piece of word) to the right. The model uses internally a mask-mechanism to make sure the predictions for the token i only uses the inputs from 1 to i but not the future tokens.

This is the smallest version of GPT-2, with 124M parameters. 

Connect to Goodle Drive and upload the dataset

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive/NLP_Project'
  sys.path.append(path_to_project)
  IN_COLAB = True
except:
  IN_COLAB = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import torch
import numpy as np

In [ ]:
dataset_path = path_to_project + '/final_ds.csv' if IN_COLAB else '/final_ds.csv'
df = pd.read_csv(dataset_path)

Import libraries that are going to be used in the following subsections

In [ ]:
!pip install -q transformers
!pip install datasets

In [ ]:
from datasets import Dataset, DatasetDict

#### Dataset Pre-processing



For the fine-tuning of gpt2-small we decided to keep 15000 samples. The training lasts approximately 2 hours, and, given the size and the pre-training of the model, gpt2 had better performance with a bigger amount of samples.

In [ ]:
df = df.head(15000).copy()
df.shape

Subset the data into training dataset, validation dataset and test dataset. We decided to use 10% of the dataset for tesing, and the remaining for train and validation.

In [ ]:
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(df, test_size=0.1)
train, val = train_test_split(train_val, test_size=0.2)

In [ ]:
print('# train instances: ', train.shape[0])
print('# test instances:  ', test.shape[0])
print('# val instances:   ', val.shape[0])

# train instances:  10800
# test instances:   1500
# val instances:    2700


Now we check the device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#### Model and Tokenizer

Retrieve the model and the tokenizer

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
model_name = 'gpt2'

Upload the Tokenizer

In [ ]:
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained(model_name)

In [ ]:
tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token

In [ ]:
print("vocabulary size: ", tokenizer_gpt2.vocab_size)

vocabulary size:  50257


In [ ]:
list(tokenizer_gpt2.get_vocab().items())[600:610]

[('int', 600),
 ('ress', 601),
 ('ations', 602),
 ('ail', 603),
 ('Ġ4', 604),
 ('ical', 605),
 ('Ġthem', 606),
 ('Ġher', 607),
 ('ount', 608),
 ('ĠCh', 609)]

In [ ]:
text = "You have an array in input, order the elements in it in O(n) time complexity. Add a wrong wordd"
encoded_input = tokenizer_gpt2._tokenize(text)
print(encoded_input)

['You', 'Ġhave', 'Ġan', 'Ġarray', 'Ġin', 'Ġinput', ',', 'Ġorder', 'Ġthe', 'Ġelements', 'Ġin', 'Ġit', 'Ġin', 'ĠO', '(', 'n', ')', 'Ġtime', 'Ġcomplexity', '.', 'ĠAdd', 'Ġa', 'Ġwrong', 'Ġword', 'd']


In [ ]:
encoded_ids = tokenizer_gpt2(text)['input_ids']
print(encoded_ids)

[1639, 423, 281, 7177, 287, 5128, 11, 1502, 262, 4847, 287, 340, 287, 440, 7, 77, 8, 640, 13357, 13, 3060, 257, 2642, 1573, 67]


It was necessary to use GPT2LMHeadModel instead of GPT2MOdel, beacuse the latter is not compatible with the Trainer due to a mismatch of the arguments.

In [ ]:
gpt2 = GPT2LMHeadModel.from_pretrained(model_name, device_map=device)
gpt2.resize_token_embeddings(len(tokenizer_gpt2))

Embedding(50257, 768)

In [ ]:
print(gpt2)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


Consider now the number of parameters of the model

In [ ]:
n_params = sum(param.numel() for param in gpt2.parameters())
n_params

124439808

In [ ]:
for name, param in gpt2.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Parameter shape: {param.size()}")
    print(f"Is trainable: {param.requires_grad}")
    print()

Parameter name: transformer.wte.weight
Parameter shape: torch.Size([50257, 768])
Is trainable: True

Parameter name: transformer.wpe.weight
Parameter shape: torch.Size([1024, 768])
Is trainable: True

Parameter name: transformer.h.0.ln_1.weight
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.ln_1.bias
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.attn.c_attn.weight
Parameter shape: torch.Size([768, 2304])
Is trainable: True

Parameter name: transformer.h.0.attn.c_attn.bias
Parameter shape: torch.Size([2304])
Is trainable: True

Parameter name: transformer.h.0.attn.c_proj.weight
Parameter shape: torch.Size([768, 768])
Is trainable: True

Parameter name: transformer.h.0.attn.c_proj.bias
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.ln_2.weight
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.ln_2.bias
Parameter shape: torch.Size([7

#### Training


Now that we have both Tokenizer and Model we can tokenize the dataset and train the model

The eos tokens have to be added by hand to obtain a single string for each 'conversation'. In a single string we include the user input, that includes the description of the problem, and the time and space complexities, and the espected answer from the assistant.

In [ ]:
def apply_eos_token(idx, df, eos_token):
    # build the user input including the problem description, the time complexity and the space complexity
    chat_string = 'User:' + df.loc[idx, 'problem_description'] + ' Time complexity: ' + df.loc[idx, 'time_complexity_inferred'] + '; Space complexity: ' + df.loc[idx, 'space_complexity_inferred']
    # now add the eos token and the response from the assistant, the code solution for the problem
    chat_string = chat_string + eos_token + 'Assistant: ' + df.loc[idx, 'solution_code'] + eos_token
    return chat_string

In [ ]:
train_str = [apply_eos_token(idx, train, tokenizer_gpt2.eos_token) for idx in train.index]
test_str = [apply_eos_token(idx, test, tokenizer_gpt2.eos_token) for idx in test.index]
val_str = [apply_eos_token(idx, val, tokenizer_gpt2.eos_token) for idx in val.index]

In [ ]:
train_str[0]

"User: Naruto has sneaked into the Orochimaru's lair and is now looking for Sasuke. There are T rooms there. Every room has a door into it, each door can be described by the number n of seals on it and their integer energies a_1, a_2, ..., a_n. All energies a_i are nonzero and do not exceed 100 by absolute value. Also, n is even.\n\nIn order to open a door, Naruto must find such n seals with integer energies b_1, b_2, ..., b_n that the following equality holds: a_{1} ⋅ b_{1} + a_{2} ⋅ b_{2} + ... + a_{n} ⋅ b_{n} = 0. All b_i must be nonzero as well as a_i are, and also must not exceed 100 by absolute value. Please find required seals for every room there.\n\nInput\n\nThe first line contains the only integer T (1 ≤ T ≤ 1000) standing for the number of rooms in the Orochimaru's lair. The other lines contain descriptions of the doors.\n\nEach description starts with the line containing the only even integer n (2 ≤ n ≤ 100) denoting the number of seals.\n\nThe following line contains the s

The strings where the eos token was applied, are now put in a dictionary structure

In [ ]:
train_data = Dataset.from_dict({'chat': train_str})
test_data = Dataset.from_dict({'chat': test_str})
val_data = Dataset.from_dict({'chat': val_str})

In [ ]:
data = DatasetDict()
data['train'] = train_data
data['val'] = val_data
data['test'] = test_data

Tokenize the data

In [ ]:
def tokenize_function(examples):
    input_encodings = tokenizer_gpt2(examples["chat"],
        truncation=True,
        padding="max_length",
        max_length=512)
    sample = {
        'input_ids': input_encodings.input_ids,
        'attention_mask': input_encodings.attention_mask,
        'labels': input_encodings.input_ids.copy()
    }
    return sample

tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/10800 [00:00<?, ? examples/s]

Map:   0%|          | 0/2700 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
# get all sequences in same batch
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer_gpt2, mlm=False)

Now the training starts

#### Calculate Perplexity before fine-tuning.

Get a random conversation from the test set

In [ ]:
import random

random.seed(43)

idx = random.choice(range(len(test_data))) # select a random conversation
print(idx)
dialogue = test_data['chat'][idx]
print(dialogue)

78
User: Pavel has several sticks with lengths equal to powers of two.

He has a_0 sticks of length 2^0 = 1, a_1 sticks of length 2^1 = 2, ..., a_{n-1} sticks of length 2^{n-1}. 

Pavel wants to make the maximum possible number of triangles using these sticks. The triangles should have strictly positive area, each stick can be used in at most one triangle.

It is forbidden to break sticks, and each triangle should consist of exactly three sticks.

Find the maximum possible number of triangles.

Input

The first line contains a single integer n (1 ≤ n ≤ 300 000) — the number of different lengths of sticks.

The second line contains n integers a_0, a_1, ..., a_{n-1} (1 ≤ a_i ≤ 10^9), where a_i is the number of sticks with the length equal to 2^i.

Output

Print a single integer — the maximum possible number of non-degenerate triangles that Pavel can make.

Examples

Input


5
1 2 2 2 2


Output


3


Input


3
1 1 1


Output


0


Input


3
3 3 3


Output


3

Note

In the first example,

In [ ]:
# now take only the 'input part' and the 'output part'
# parse string
test_input, test_output, end = dialogue.split('<|endoftext|>')

test_input = test_input + tokenizer_gpt2.eos_token + ' Assistant: '
test_output = test_output + tokenizer_gpt2.eos_token

print('User input: ', test_input)
print('####')
print('Correct solution output: ', test_output)

User input:  User: Pavel has several sticks with lengths equal to powers of two.

He has a_0 sticks of length 2^0 = 1, a_1 sticks of length 2^1 = 2, ..., a_{n-1} sticks of length 2^{n-1}. 

Pavel wants to make the maximum possible number of triangles using these sticks. The triangles should have strictly positive area, each stick can be used in at most one triangle.

It is forbidden to break sticks, and each triangle should consist of exactly three sticks.

Find the maximum possible number of triangles.

Input

The first line contains a single integer n (1 ≤ n ≤ 300 000) — the number of different lengths of sticks.

The second line contains n integers a_0, a_1, ..., a_{n-1} (1 ≤ a_i ≤ 10^9), where a_i is the number of sticks with the length equal to 2^i.

Output

Print a single integer — the maximum possible number of non-degenerate triangles that Pavel can make.

Examples

Input


5
1 2 2 2 2


Output


3


Input


3
1 1 1


Output


0


Input


3
3 3 3


Output


3

Note

In the firs

We decided to use pipeline from the transformers library to generate the output of the model, indicating the task 'text-generation'

In [ ]:
!pip install pipeline transformers evaluate
from transformers import pipeline
import evaluate

In [ ]:
pipe = pipeline("text-generation", model= gpt2, tokenizer=tokenizer_gpt2)

Device set to use cuda


In [ ]:
gen_output = pipe(test_input, max_new_tokens=256)
print("Generated text:\n", gen_output[0]["generated_text"])

Generated text:
 User: Pavel has several sticks with lengths equal to powers of two.

He has a_0 sticks of length 2^0 = 1, a_1 sticks of length 2^1 = 2, ..., a_{n-1} sticks of length 2^{n-1}. 

Pavel wants to make the maximum possible number of triangles using these sticks. The triangles should have strictly positive area, each stick can be used in at most one triangle.

It is forbidden to break sticks, and each triangle should consist of exactly three sticks.

Find the maximum possible number of triangles.

Input

The first line contains a single integer n (1 ≤ n ≤ 300 000) — the number of different lengths of sticks.

The second line contains n integers a_0, a_1, ..., a_{n-1} (1 ≤ a_i ≤ 10^9), where a_i is the number of sticks with the length equal to 2^i.

Output

Print a single integer — the maximum possible number of non-degenerate triangles that Pavel can make.

Examples

Input


5
1 2 2 2 2


Output


3


Input


3
1 1 1


Output


0


Input


3
3 3 3


Output


3

Note

In the 

The generated text has nothing to do with python code and includes references to pop culture, probably included in the pre-training of the model

**Perplexity**

In [ ]:
# get inputs from test_data
test_input = [dialogue.split('<|endoftext|>')[0] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

# get outputs from test data
test_output = [dialogue.split('<|endoftext|>')[1] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

In [ ]:
perplexity_metric = evaluate.load("perplexity", module_type="metric")
perplexity_before_finetuning = perplexity_metric.compute(
    predictions= test_input,
    add_start_token=False,
    model_id= 'gpt2'
)
print("Perplexity before fine-tuning:", perplexity_before_finetuning)

  0%|          | 0/1 [00:00<?, ?it/s]

Perplexity before fine-tuning: {'perplexities': [28.182695388793945, 14.845183372497559, 14.715679168701172], 'mean_perplexity': np.float64(19.24785264333089)}


#### Fine-Tuning
Now we can start the fine-tuning

In [ ]:
from transformers import TrainingArguments, Trainer
import os

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = TrainingArguments(
    "gpt2_trainer",
    #evaluation_strategy="steps",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8, 
    num_train_epochs=3,
    save_steps=500,
    eval_steps=500,
    learning_rate=6.25e-5,
    lr_scheduler_type="linear",
    bf16=True,  
    report_to=None,  
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Trainer(
    model= gpt2,
    args= training_args,
    train_dataset= tokenized_data['train'],
    eval_dataset= tokenized_data['val'],
    data_collator= data_collator
)

Measure the time taken by the model to finish training

In [ ]:
import time
begin = time.time()

In [ ]:
trainer.train()

Step,Training Loss
500,1.133900
1000,0.341100
1500,0.236900
2000,0.211400


TrainOutput(global_step=2025, training_loss=0.4775619372615108, metrics={'train_runtime': 7315.882, 'train_samples_per_second': 4.429, 'train_steps_per_second': 0.277, 'total_flos': 8465861836800000.0, 'train_loss': 0.4775619372615108, 'epoch': 3.0})

In [ ]:
end = time.time()
print("Training time: ", end - begin)

Training time:  7316.4667019844055


In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(gpt2.named_parameters())

print('The GPT-2 model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:2]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[2:14]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-2:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The GPT-2 model has 148 different named parameters.

==== Embedding Layer ====

transformer.wte.weight                                  (50257, 768)
transformer.wpe.weight                                   (1024, 768)

==== First Transformer ====

transformer.h.0.ln_1.weight                                   (768,)
transformer.h.0.ln_1.bias                                     (768,)
transformer.h.0.attn.c_attn.weight                       (768, 2304)
transformer.h.0.attn.c_attn.bias                             (2304,)
transformer.h.0.attn.c_proj.weight                        (768, 768)
transformer.h.0.attn.c_proj.bias                              (768,)
transformer.h.0.ln_2.weight                                   (768,)
transformer.h.0.ln_2.bias                                     (768,)
transformer.h.0.mlp.c_fc.weight                          (768, 3072)
transformer.h.0.mlp.c_fc.bias                                (3072,)
transformer.h.0.mlp.c_proj.weight                        (3072

In [ ]:
gpt2.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

Save model's parameters

In [ ]:
from datetime import datetime

gpt2_finetune_path = path_to_project + '/Transformer-trained-models/' + f"gpt2_fine_tuning_(chat+code)_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"
tokenizer_gpt2.save_pretrained(gpt2_finetune_path)
gpt2.save_pretrained(gpt2_finetune_path)
print(f"Checkpoint saved at: \'{gpt2_finetune_path}\'")

Checkpoint saved at: '/content/drive/MyDrive/UnB/NLP_Project/Transformer-trained-models/gpt2_fine_tuning_(chat+code)_2025_05_15_13_50_58'


#### Testing

Retrieve the trained model

In [ ]:
!pip install evaluate
import evaluate

In [ ]:
device = 'cuda'

gpt2_finetune_path = '/content/drive/MyDrive/NLP_Project/Transformer-trained-models/gpt2_fine_tuning_(chat+code)_2025_05_15_13_50_58'
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained(gpt2_finetune_path)
gpt2 = GPT2LMHeadModel.from_pretrained(gpt2_finetune_path, device_map=device)

To test the model, we first extract one chat from the test data randomly, give it as input to the model, and we see the response

In [ ]:
import random

random.seed(43)

idx = random.choice(range(len(test_data))) # select a random conversation
print(idx)
dialogue = test_data['chat'][idx]
print(dialogue)

78
User: Pavel has several sticks with lengths equal to powers of two.

He has a_0 sticks of length 2^0 = 1, a_1 sticks of length 2^1 = 2, ..., a_{n-1} sticks of length 2^{n-1}. 

Pavel wants to make the maximum possible number of triangles using these sticks. The triangles should have strictly positive area, each stick can be used in at most one triangle.

It is forbidden to break sticks, and each triangle should consist of exactly three sticks.

Find the maximum possible number of triangles.

Input

The first line contains a single integer n (1 ≤ n ≤ 300 000) — the number of different lengths of sticks.

The second line contains n integers a_0, a_1, ..., a_{n-1} (1 ≤ a_i ≤ 10^9), where a_i is the number of sticks with the length equal to 2^i.

Output

Print a single integer — the maximum possible number of non-degenerate triangles that Pavel can make.

Examples

Input


5
1 2 2 2 2


Output


3


Input


3
1 1 1


Output


0


Input


3
3 3 3


Output


3

Note

In the first example,

In [ ]:
# now take only the 'input part' and the 'output part'
# parse string
test_input, test_output, end = dialogue.split('<|endoftext|>')

# add eos token at the end of test_input and test_output
test_input = test_input + tokenizer_gpt2.eos_token + ' Assistant: '
test_output = test_output + tokenizer_gpt2.eos_token

print('User input: \n', test_input)
print('####')
print('Correct solution output: \n', test_output)

User input: 
 User: Pavel has several sticks with lengths equal to powers of two.

He has a_0 sticks of length 2^0 = 1, a_1 sticks of length 2^1 = 2, ..., a_{n-1} sticks of length 2^{n-1}. 

Pavel wants to make the maximum possible number of triangles using these sticks. The triangles should have strictly positive area, each stick can be used in at most one triangle.

It is forbidden to break sticks, and each triangle should consist of exactly three sticks.

Find the maximum possible number of triangles.

Input

The first line contains a single integer n (1 ≤ n ≤ 300 000) — the number of different lengths of sticks.

The second line contains n integers a_0, a_1, ..., a_{n-1} (1 ≤ a_i ≤ 10^9), where a_i is the number of sticks with the length equal to 2^i.

Output

Print a single integer — the maximum possible number of non-degenerate triangles that Pavel can make.

Examples

Input


5
1 2 2 2 2


Output


3


Input


3
1 1 1


Output


0


Input


3
3 3 3


Output


3

Note

In the fir

In [ ]:
!pip install pipeline transformers
from transformers import pipeline

pipe_finetuned = pipeline("text-generation", model= gpt2, tokenizer=tokenizer_gpt2)

In [ ]:
gen_output = pipe_finetuned(test_input, max_new_tokens=256)
print("Generated text after the fine-tuning: \n", gen_output[0]["generated_text"])

Generated text after the fine-tuning: 
 User: Pavel has several sticks with lengths equal to powers of two.

He has a_0 sticks of length 2^0 = 1, a_1 sticks of length 2^1 = 2, ..., a_{n-1} sticks of length 2^{n-1}. 

Pavel wants to make the maximum possible number of triangles using these sticks. The triangles should have strictly positive area, each stick can be used in at most one triangle.

It is forbidden to break sticks, and each triangle should consist of exactly three sticks.

Find the maximum possible number of triangles.

Input

The first line contains a single integer n (1 ≤ n ≤ 300 000) — the number of different lengths of sticks.

The second line contains n integers a_0, a_1, ..., a_{n-1} (1 ≤ a_i ≤ 10^9), where a_i is the number of sticks with the length equal to 2^i.

Output

Print a single integer — the maximum possible number of non-degenerate triangles that Pavel can make.

Examples

Input


5
1 2 2 2 2


Output


3


Input


3
1 1 1


Output


0


Input


3
3 3 3


Ou

As it can be noticed, after the fine-tuning the model is able to generate python code. The generated text starts with a reference to the user input, but it able to go forward and include an answer from the 'Assistant'. The model is not yet able to generate a correct solution for the problem, but given the smaller size of gpt2-small, it was expected.

#### Evaluation Metrics

It was decided to consider the metrics Perplexity and BLEU.

**Perplexity**

In [ ]:
# get inputs from test_data
test_input = [dialogue.split('<|endoftext|>')[0] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

# get outputs from test data
test_output = [dialogue.split('<|endoftext|>')[1] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

In [ ]:
perplexity_metric = evaluate.load("perplexity", module_type="metric")
perplexity = perplexity_metric.compute(
    predictions= test_input,
    add_start_token=False,
    model_id= gpt2_finetune_path
)
print("Perplexity post fine-tuning:", perplexity)

  0%|          | 0/1 [00:00<?, ?it/s]

Perplexity post fine-tuning: {'perplexities': [1.0604777336120605, 1.047598958015442, 1.0766164064407349], 'mean_perplexity': np.float64(1.0615643660227458)}


The perplexity calculated before the fine-tuning was [28.182695388793945, 14.845183372497559, 14.715679168701172] with mean_perplexity: np.float64(19.24785264333089)

We can see, now, that the perplexity values have sistematically decreased, indicating that the fine-tuning  process was able to improve the performance of the model on this specific task.

**BLEU**

BLEU focuses on precision by counting matching n-grams.

In [ ]:
# BLEU Score
bleu_metric = evaluate.load("bleu")
bleu_results = bleu_metric.compute(
    predictions= [pipe_finetuned(test_input[0], max_new_tokens=256)[0]['generated_text']], 
    references= [test_output[0]]  
)
print("BLEU Score:", bleu_results)

BLEU Score: {'bleu': 0.12839728594651886, 'precisions': [0.3142857142857143, 0.14537444933920704, 0.09271523178807947, 0.06415929203539823], 'brevity_penalty': 1.0, 'length_ratio': 1.6666666666666667, 'translation_length': 455, 'reference_length': 273}


The Bleu scores are still low overall, but are much higher than the Bleu scores for the T5 models, further indicating how the better performance among similar sized model was achieved by the gpt2 model

### Fine-tuning gpt2-CodeParrot

This model is a base GPT-2 architecture with 124M parameters. It was trained on the huggingface-course/codeparrot-ds-valid dataset, which is a small subset of the original WebText dataset used to train GPT-2, and includes python code.

Connect to Goodle Drive and upload the dataset

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive/NLP_Project'
  sys.path.append(path_to_project)
  IN_COLAB = True
except:
  IN_COLAB = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import torch
import numpy as np

In [ ]:
dataset_path = path_to_project + '/final_ds.csv' if IN_COLAB else '/final_ds.csv'
df = pd.read_csv(dataset_path)

Import libraries that are going to be used in the following subsections

In [ ]:
!pip install -q transformers
!pip install datasets

In [ ]:
from datasets import Dataset, DatasetDict

#### Dataset Pre-processing



Considering the small size of the model and the limited data used during its training, we decided to reduce the number of samples used for the fine-tuning to avoid overwriting parameters unecessarily and degrade the performance of the model

In [ ]:
df = df.head(5000).copy()
df.shape

Subset the data into training dataset, validation dataset and test dataset. We decided to use 10% of the dataset for tesing, and the remaining for train and validation.

In [ ]:
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(df, test_size=0.1)
train, val = train_test_split(train_val, test_size=0.2)

In [ ]:
print('# train instances: ', train.shape[0])
print('# test instances:  ', test.shape[0])
print('# val instances:   ', val.shape[0])

# train instances:  3600
# test instances:   500
# val instances:    900


Now we check the device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#### Model and Tokenizer

Retreiev the model and the tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = 'kailasps/GPT2-codeparrot'

Upload the Tokenizer

In [ ]:
tokenizer_gpt2 = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/789k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/448k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

In [ ]:
tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token

In [ ]:
print("vocabulary size: ", tokenizer_gpt2.vocab_size)

vocabulary size:  50000


In [ ]:
list(tokenizer_gpt2.get_vocab().items())[600:610]

[('ĠSources', 46176),
 ('Ġcac', 33927),
 ('Ġsegm', 31074),
 ('electric', 30949),
 ('Ġuntagged', 43874),
 ('ellipse', 22936),
 ('ĠSubscription', 26798),
 ('Ġlooping', 24598),
 ('`', 64),
 ('HierarchySession', 26069)]

In [ ]:
text = "You have an array in input, order the elements in it in O(n) time complexity. Add a wrong wordd"
encoded_input = tokenizer_gpt2.tokenize(text)
print(encoded_input)

['You', 'Ġhave', 'Ġan', 'Ġarray', 'Ġin', 'Ġinput', ',', 'Ġorder', 'Ġthe', 'Ġelements', 'Ġin', 'Ġit', 'Ġin', 'ĠO', '(', 'n', ')', 'Ġtime', 'Ġcomplexity', '.', 'ĠAdd', 'Ġa', 'Ġwrong', 'Ġword', 'd']


In [ ]:
encoded_ids = tokenizer_gpt2(text)['input_ids']
print(encoded_ids)

[6147, 1054, 309, 960, 253, 935, 12, 1444, 256, 2347, 253, 577, 253, 690, 8, 78, 9, 626, 16989, 14, 1822, 231, 6401, 1975, 68]


The model is uploaded and connected to the device


In [ ]:
gpt2 = AutoModelForCausalLM.from_pretrained(model_name, device_map=device)
gpt2.resize_token_embeddings(len(tokenizer_gpt2))

config.json:   0%|          | 0.00/898 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/497M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Embedding(50000, 768)

In [ ]:
print(gpt2)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50000, bias=False)
)


Consider now the number of parameters of the model

In [ ]:
n_params = sum(param.numel() for param in gpt2.parameters())
n_params

124242432

In [ ]:
for name, param in gpt2.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Parameter shape: {param.size()}")
    print(f"Is trainable: {param.requires_grad}")
    print()

Parameter name: transformer.wte.weight
Parameter shape: torch.Size([50000, 768])
Is trainable: True

Parameter name: transformer.wpe.weight
Parameter shape: torch.Size([1024, 768])
Is trainable: True

Parameter name: transformer.h.0.ln_1.weight
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.ln_1.bias
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.attn.c_attn.weight
Parameter shape: torch.Size([768, 2304])
Is trainable: True

Parameter name: transformer.h.0.attn.c_attn.bias
Parameter shape: torch.Size([2304])
Is trainable: True

Parameter name: transformer.h.0.attn.c_proj.weight
Parameter shape: torch.Size([768, 768])
Is trainable: True

Parameter name: transformer.h.0.attn.c_proj.bias
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.ln_2.weight
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.ln_2.bias
Parameter shape: torch.Size([7

#### Training


Now that we have both Tokenizer and Model we can tokenize the dataset and train the model

In [ ]:
def apply_eos_token(idx, df, eos_token):
    # build the user input including the problem description, the time complexity and the space complexity
    chat_string = 'User:' + df.loc[idx, 'problem_description'] + ' Time complexity: ' + df.loc[idx, 'time_complexity_inferred'] + '; Space complexity: ' + df.loc[idx, 'space_complexity_inferred']
    # now add the eos token and the response from the assistant, the code solution for the problem
    chat_string = chat_string + eos_token + 'Assistant: ' + df.loc[idx, 'solution_code'] + eos_token
    return chat_string

In [ ]:
train_str = [apply_eos_token(idx, train, tokenizer_gpt2.eos_token) for idx in train.index]
test_str = [apply_eos_token(idx, test, tokenizer_gpt2.eos_token) for idx in test.index]
val_str = [apply_eos_token(idx, val, tokenizer_gpt2.eos_token) for idx in val.index]

In [ ]:
train_str[0]

'User: Manao has invented a new mathematical term — a beautiful set of points. He calls a set of points on a plane beautiful if it meets the following conditions:\n\n  1. The coordinates of each point in the set are integers. \n  2. For any two points from the set, the distance between them is a non-integer. \n\n\n\nConsider all points (x, y) which satisfy the inequations: 0 ≤ x ≤ n; 0 ≤ y ≤ m; x + y > 0. Choose their subset of maximum size such that it is also a beautiful set of points.\n\nInput\n\nThe single line contains two space-separated integers n and m (1 ≤ n, m ≤ 100).\n\nOutput\n\nIn the first line print a single integer — the size k of the found beautiful set. In each of the next k lines print a pair of space-separated integers — the x- and y- coordinates, respectively, of a point from the set.\n\nIf there are several optimal solutions, you may print any of them.\n\nExamples\n\nInput\n\n2 2\n\n\nOutput\n\n3\n0 1\n1 2\n2 0\n\n\nInput\n\n4 3\n\n\nOutput\n\n4\n0 3\n2 1\n3 0\n4 

The strings where the eos token was applied, are now put in a dictionary structure

In [ ]:
train_data = Dataset.from_dict({'chat': train_str})
test_data = Dataset.from_dict({'chat': test_str})
val_data = Dataset.from_dict({'chat': val_str})

In [ ]:
data = DatasetDict()
data['train'] = train_data
data['val'] = val_data
data['test'] = test_data

Tokenize the data

In [ ]:
def tokenize_function(examples):
    input_encodings = tokenizer_gpt2(examples["chat"],
        truncation=True,
        padding="max_length",
        max_length=512)
    sample = {
        'input_ids': input_encodings.input_ids,
        'attention_mask': input_encodings.attention_mask,
        'labels': input_encodings.input_ids.copy()
    }
    return sample

tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# get all sequences in same batch
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer_gpt2, mlm=False)

Now the training starts

##### Calculate Perplexity before fine-tuning.

Get a random conversation from the test set

In [ ]:
import random

random.seed(43)

idx = random.choice(range(len(test_data))) # select a random conversation
print(idx)
dialogue = test_data['chat'][idx]
print(dialogue)

19
User: Bob is playing with 6-sided dice. A net of such standard cube is shown below.

<image>

He has an unlimited supply of these dice and wants to build a tower by stacking multiple dice on top of each other, while choosing the orientation of each dice. Then he counts the number of visible pips on the faces of the dice.

For example, the number of visible pips on the tower below is 29 — the number visible on the top is 1, from the south 5 and 3, from the west 4 and 2, from the north 2 and 4 and from the east 3 and 5.

<image>

The one at the bottom and the two sixes by which the dice are touching are not visible, so they are not counted towards total.

Bob also has t favourite integers x_i, and for every such integer his goal is to build such a tower that the number of visible pips is exactly x_i. For each of Bob's favourite integers determine whether it is possible to build a tower that has exactly that many visible pips.

Input

The first line contains a single integer t (1 ≤ t ≤

In [ ]:
# now take only the 'input part' and the 'output part'
# parse string
test_input, test_output, end = dialogue.split('<|endoftext|>')

test_input = test_input + tokenizer_gpt2.eos_token + ' Assistant: '
test_output = test_output + tokenizer_gpt2.eos_token

print('User input: ', test_input)
print('####')
print('Correct solution output: ', test_output)

User input:  User: Bob is playing with 6-sided dice. A net of such standard cube is shown below.

<image>

He has an unlimited supply of these dice and wants to build a tower by stacking multiple dice on top of each other, while choosing the orientation of each dice. Then he counts the number of visible pips on the faces of the dice.

For example, the number of visible pips on the tower below is 29 — the number visible on the top is 1, from the south 5 and 3, from the west 4 and 2, from the north 2 and 4 and from the east 3 and 5.

<image>

The one at the bottom and the two sixes by which the dice are touching are not visible, so they are not counted towards total.

Bob also has t favourite integers x_i, and for every such integer his goal is to build such a tower that the number of visible pips is exactly x_i. For each of Bob's favourite integers determine whether it is possible to build a tower that has exactly that many visible pips.

Input

The first line contains a single integer 

In [ ]:
!pip install pipeline transformers evaluate
from transformers import pipeline
import evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00


In [ ]:
pipe = pipeline("text-generation", model= gpt2, tokenizer=tokenizer_gpt2)

Device set to use cuda


In [ ]:
gen_output = pipe(test_input, max_new_tokens=256)
print("Generated text:\n", gen_output[0]["generated_text"])

Generated text:
 User: Bob is playing with 6-sided dice. A net of such standard cube is shown below.

<image>

He has an unlimited supply of these dice and wants to build a tower by stacking multiple dice on top of each other, while choosing the orientation of each dice. Then he counts the number of visible pips on the faces of the dice.

For example, the number of visible pips on the tower below is 29 — the number visible on the top is 1, from the south 5 and 3, from the west 4 and 2, from the north 2 and 4 and from the east 3 and 5.

<image>

The one at the bottom and the two sixes by which the dice are touching are not visible, so they are not counted towards total.

Bob also has t favourite integers x_i, and for every such integer his goal is to build such a tower that the number of visible pips is exactly x_i. For each of Bob's favourite integers determine whether it is possible to build a tower that has exactly that many visible pips.

Input

The first line contains a single inte

The generated text does not include the solution code

**Perplexity**

In [ ]:
# get inputs from test_data
test_input = [dialogue.split('<|endoftext|>')[0] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

# get outputs from test data
test_output = [dialogue.split('<|endoftext|>')[1] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

In [ ]:
perplexity_metric = evaluate.load("perplexity", module_type="metric")
perplexity_before_finetuning = perplexity_metric.compute(
    predictions= test_input,
    model_id= model_name
)
print("Perplexity before fine-tuning:", perplexity_before_finetuning)

  0%|          | 0/1 [00:00<?, ?it/s]

Perplexity before fine-tuning: {'perplexities': [653.0138549804688, 570.7035522460938, 424.50201416015625], 'mean_perplexity': np.float64(549.4064737955729)}


#### Fine-Tuning
Now we can start the fine-tuning

In [ ]:
from transformers import TrainingArguments, Trainer
import os

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = TrainingArguments(
    "gpt2-parrot_trainer",
    #evaluation_strategy="steps",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8, 
    num_train_epochs=3,
    save_steps=500,
    eval_steps=500,
    learning_rate=6.25e-5,
    lr_scheduler_type="linear",
    bf16=True,  
    report_to=None, 
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Trainer(
    model= gpt2,
    args= training_args,
    train_dataset= tokenized_data['train'],
    eval_dataset= tokenized_data['val'],
    data_collator= data_collator
)

Measure the time taken by the model to finish training

In [ ]:
import time
begin = time.time()

In [ ]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,1.458700


TrainOutput(global_step=675, training_loss=1.1649107078269676, metrics={'train_runtime': 2385.8294, 'train_samples_per_second': 4.527, 'train_steps_per_second': 0.283, 'total_flos': 2821953945600000.0, 'train_loss': 1.1649107078269676, 'epoch': 3.0})

In [ ]:
end = time.time()
print("Training time: ", end - begin)

Training time:  2386.2620153427124


In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(gpt2.named_parameters())

print('The GPT-2 model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:2]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[2:14]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-2:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The GPT-2 model has 148 different named parameters.

==== Embedding Layer ====

transformer.wte.weight                                  (50000, 768)
transformer.wpe.weight                                   (1024, 768)

==== First Transformer ====

transformer.h.0.ln_1.weight                                   (768,)
transformer.h.0.ln_1.bias                                     (768,)
transformer.h.0.attn.c_attn.weight                       (768, 2304)
transformer.h.0.attn.c_attn.bias                             (2304,)
transformer.h.0.attn.c_proj.weight                        (768, 768)
transformer.h.0.attn.c_proj.bias                              (768,)
transformer.h.0.ln_2.weight                                   (768,)
transformer.h.0.ln_2.bias                                     (768,)
transformer.h.0.mlp.c_fc.weight                          (768, 3072)
transformer.h.0.mlp.c_fc.bias                                (3072,)
transformer.h.0.mlp.c_proj.weight                        (3072

In [ ]:
gpt2.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50000, bias=False)
)

Save model's parameters

In [ ]:
from datetime import datetime

gpt2_finetune_path = path_to_project + '/Transformer-trained-models/' + f"gpt2-parrot_fine_tuning_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"
tokenizer_gpt2.save_pretrained(gpt2_finetune_path)
gpt2.save_pretrained(gpt2_finetune_path)
print(f"Checkpoint saved at: \'{gpt2_finetune_path}\'")

Checkpoint saved at: '/content/drive/MyDrive/SFX/NLP_Project/Transformer-trained-models/gpt2-parrot_fine_tuning_2025_05_16_14_58_31'


#### Testing

Retrieve the trained model

In [ ]:
!pip install evaluate
import evaluate

In [ ]:
device = 'cuda'

#gpt2_finetune_path = '/content/drive/MyDrive/NLP_Project/Transformer-trained-models/gpt2-parrot_fine_tuning_2025_05_16_14_58_31'
tokenizer_gpt2 = AutoTokenizer.from_pretrained(gpt2_finetune_path)
gpt2 = AutoModelForCausalLM.from_pretrained(gpt2_finetune_path, device_map = device)

To test the model, we first extract one chat from the test data randomly, give it as input to the model, and we see the response

In [ ]:
import random

random.seed(43)

idx = random.choice(range(len(test_data))) # select a random conversation
print(idx)
dialogue = test_data['chat'][idx]
print(dialogue)

19
User: Bob is playing with 6-sided dice. A net of such standard cube is shown below.

<image>

He has an unlimited supply of these dice and wants to build a tower by stacking multiple dice on top of each other, while choosing the orientation of each dice. Then he counts the number of visible pips on the faces of the dice.

For example, the number of visible pips on the tower below is 29 — the number visible on the top is 1, from the south 5 and 3, from the west 4 and 2, from the north 2 and 4 and from the east 3 and 5.

<image>

The one at the bottom and the two sixes by which the dice are touching are not visible, so they are not counted towards total.

Bob also has t favourite integers x_i, and for every such integer his goal is to build such a tower that the number of visible pips is exactly x_i. For each of Bob's favourite integers determine whether it is possible to build a tower that has exactly that many visible pips.

Input

The first line contains a single integer t (1 ≤ t ≤

In [ ]:
# now take only the 'input part' and the 'output part'
# parse string
test_input, test_output, end = dialogue.split('<|endoftext|>')

# add eos token at the end of test_input and test_output
test_input = test_input + tokenizer_gpt2.eos_token
test_output = test_output + tokenizer_gpt2.eos_token

print('User input: \n', test_input)
print('####')
print('Correct solution output: \n', test_output)

User input: 
 User: Bob is playing with 6-sided dice. A net of such standard cube is shown below.

<image>

He has an unlimited supply of these dice and wants to build a tower by stacking multiple dice on top of each other, while choosing the orientation of each dice. Then he counts the number of visible pips on the faces of the dice.

For example, the number of visible pips on the tower below is 29 — the number visible on the top is 1, from the south 5 and 3, from the west 4 and 2, from the north 2 and 4 and from the east 3 and 5.

<image>

The one at the bottom and the two sixes by which the dice are touching are not visible, so they are not counted towards total.

Bob also has t favourite integers x_i, and for every such integer his goal is to build such a tower that the number of visible pips is exactly x_i. For each of Bob's favourite integers determine whether it is possible to build a tower that has exactly that many visible pips.

Input

The first line contains a single integer

In [ ]:
!pip install pipeline transformers evaluate
from transformers import pipeline
import evaluate
pipe_finetuned = pipeline("text-generation", model= gpt2, tokenizer=tokenizer_gpt2)

In [ ]:
gen_output = pipe_finetuned(test_input, max_new_tokens=256)
print("Generated text after the fine-tuning: \n", gen_output[0]["generated_text"])

Generated text after the fine-tuning: 
 User: Bob is playing with 6-sided dice. A net of such standard cube is shown below.

<image>

He has an unlimited supply of these dice and wants to build a tower by stacking multiple dice on top of each other, while choosing the orientation of each dice. Then he counts the number of visible pips on the faces of the dice.

For example, the number of visible pips on the tower below is 29 — the number visible on the top is 1, from the south 5 and 3, from the west 4 and 2, from the north 2 and 4 and from the east 3 and 5.

<image>

The one at the bottom and the two sixes by which the dice are touching are not visible, so they are not counted towards total.

Bob also has t favourite integers x_i, and for every such integer his goal is to build such a tower that the number of visible pips is exactly x_i. For each of Bob's favourite integers determine whether it is possible to build a tower that has exactly that many visible pips.

Input

The first line

The generated text after the fine-tuning includes now the python code. The code is still not correct, especially from a syntactical point of view, but the model is able to maintain the espected format of the response ('user:' and 'assistant:')

#### Evaluation Metrics

It was decided to consider the metrics Perplexity and BLEU.

**Perplexity**

In [ ]:
# get inputs from test_data
test_input = [dialogue.split('<|endoftext|>')[0] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

# get outputs from test data
test_output = [dialogue.split('<|endoftext|>')[1] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

In [ ]:
perplexity_metric = evaluate.load("perplexity", module_type="metric")
perplexity = perplexity_metric.compute(
    predictions= test_input,
    add_start_token=False,
    model_id= gpt2_finetune_path
)
print("Perplexity post fine-tuning:", perplexity)

  0%|          | 0/1 [00:00<?, ?it/s]

Perplexity post fine-tuning: {'perplexities': [1.080601692199707, 1.1230411529541016, 1.0684187412261963], 'mean_perplexity': np.float64(1.0906871954600017)}


The perplexity calculated before the fine-tuning was [653.0138549804688, 570.7035522460938, 424.50201416015625] with mean_perplexity: np.float64(549.4064737955729)

Now we can notice how the values for the perplexity are much lower, similar to those achieved by the gpt2-small model, indicating how the fine-tuning process managed to improve the performance.

**BLEU**

BLEU focuses on precision by counting matching n-grams.

In [ ]:
# BLEU Score
bleu_metric = evaluate.load("bleu")
bleu_results = bleu_metric.compute(
    predictions= [pipe_finetuned(test_input[0], max_new_tokens=256)[0]['generated_text']], 
    references= [test_output[0]]  
)
print("BLEU Score:", bleu_results)

BLEU Score: {'bleu': 0.05315658078526119, 'precisions': [0.3118081180811808, 0.10351201478743069, 0.03333333333333333, 0.0074211502782931356], 'brevity_penalty': 1.0, 'length_ratio': 1.7828947368421053, 'translation_length': 542, 'reference_length': 304}


The Bleu score, even if higher than the scores achieved by the T5 models, are still low, indicating that the generated text is still far from the reference text. 